## PyTorch 基础 : 神经网络包nn和优化器optm
torch.nn是专门为神经网络设计的模块化接口。nn构建于 Autograd之上，可用来定义和运行神经网络。 这里我们主要介绍几个一些常用的类

**约定：torch.nn 我们为了方便使用，会为他设置别名为nn，本章除nn以外还有其他的命名约定**

In [12]:

# 首先要引入相关的包
import torch
# 引入torch.nn并指定别名
import torch.nn as nn
#打印一下版本
torch.__version__

'1.3.1+cpu'

除了nn别名以外，我们还引用了nn.functional，这个包中包含了神经网络中使用的一些常用函数，这些函数的特点是，不具有可学习的参数(如ReLU，pool，DropOut等)，这些函数可以放在构造函数中，也可以不放，但是这里建议不放。

一般情况下我们会**将nn.functional 设置为大写的F**，这样缩写方便调用

In [13]:
import torch.nn.functional as F

## 定义一个网络
PyTorch中已经为我们准备好了现成的网络模型，只要继承nn.Module，并实现它的forward方法，PyTorch会根据autograd，自动实现backward函数，在forward函数中可使用任何tensor支持的函数，还可以使用if、for循环、print、log等Python语法，写法和标准的Python写法一致。

In [14]:
class Net(nn.Module):
    def __init__(self):
        # nn.Module子类的函数必须在构造函数中执行父类的构造函数
        super(Net, self).__init__()
        
        # 卷积层 '1'表示输入图片为单通道， '6'表示输出通道数，'3'表示卷积核为3*3
        self.conv1 = nn.Conv2d(1, 6, 3) 
        #线性层，输入1350个特征，输出10个特征
        self.fc1   = nn.Linear(1350, 10)  #这里的1350是如何计算的呢？这就要看后面的forward函数
    #正向传播 
    def forward(self, x): 
        print(x.size()) # 结果：[1, 1, 32, 32]
        # 卷积 -> 激活 -> 池化 
        x = self.conv1(x) #根据卷积的尺寸计算公式，计算结果是30，具体计算公式后面第二章第四节 卷积神经网络 有详细介绍。
        x = F.relu(x)
        print(x.size()) # 结果：[1, 6, 30, 30]
        x = F.max_pool2d(x, (2, 2)) #我们使用池化层，计算结果是15
        x = F.relu(x)
        print(x.size()) # 结果：[1, 6, 15, 15]
        # reshape，‘-1’表示自适应
        #这里做的就是压扁的操作 就是把后面的[1, 6, 15, 15]压扁，变为 [1, 1350]
        x = x.view(x.size()[0], -1) 
        print(x.size()) # 这里就是fc1层的的输入1350 
        x = self.fc1(x)        
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1350, out_features=10, bias=True)
)


网络的可学习参数通过net.parameters()返回

In [15]:
for parameters in net.parameters():
    print(parameters)

Parameter containing:
tensor([[[[ 0.0219, -0.2573,  0.0889],
          [-0.1050, -0.2228,  0.0377],
          [-0.2784,  0.1699,  0.0314]]],


        [[[ 0.2185,  0.1845, -0.0585],
          [ 0.1669,  0.2235,  0.1564],
          [ 0.1872, -0.1624, -0.0174]]],


        [[[ 0.1373,  0.1286, -0.3258],
          [-0.2105,  0.1048, -0.0169],
          [ 0.0384, -0.0350,  0.1084]]],


        [[[-0.1642, -0.2460, -0.2138],
          [-0.3141,  0.3321,  0.1806],
          [-0.1315,  0.0897, -0.1906]]],


        [[[ 0.2599, -0.2280, -0.1790],
          [-0.0508, -0.3046, -0.1980],
          [-0.3061, -0.0714, -0.1342]]],


        [[[ 0.0590, -0.0618, -0.2489],
          [-0.2582,  0.2022, -0.2157],
          [-0.2936, -0.2089, -0.3008]]]], requires_grad=True)
Parameter containing:
tensor([ 0.1413, -0.1041, -0.1078,  0.1417, -0.2575, -0.1221],
       requires_grad=True)
Parameter containing:
tensor([[ 6.5849e-03, -2.2716e-02,  6.8537e-03,  ..., -8.0602e-03,
         -2.3626e-02,  2.2081e-0

net.named_parameters可同时返回可学习的参数及名称。

In [16]:
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 3, 3])
conv1.bias : torch.Size([6])
fc1.weight : torch.Size([10, 1350])
fc1.bias : torch.Size([10])


forward函数的输入和输出都是Tensor

In [17]:
input = torch.randn(1, 1, 32, 32) # 这里的对应前面fforward的输入是32
out = net(input)
out.size()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


torch.Size([1, 10])

In [18]:
input.size()

torch.Size([1, 1, 32, 32])

在反向传播前，先要将所有参数的梯度清零

In [19]:
net.zero_grad() 
out.backward(torch.ones(1,10)) # 反向传播的实现是PyTorch自动实现的，我们只要调用这个函数即可

**注意**:torch.nn只支持mini-batches，不支持一次只输入一个样本，即一次必须是一个batch。

也就是说，就算我们输入一个样本，也会对样本进行分批，所以，所有的输入都会增加一个维度，我们对比下刚才的input，nn中定义为3维，但是我们人工创建时多增加了一个维度，变为了4维，最前面的1即为batch-size

## 损失函数
在nn中PyTorch还预制了常用的损失函数，下面我们用MSELoss用来计算均方误差

In [20]:
y = torch.arange(0,10).view(1,10).float()
criterion = nn.MSELoss()
loss = criterion(out, y)
#loss是个scalar，我们可以直接用item获取到他的python类型的数值
print(loss.item())

28.388608932495117


## 优化器
在反向传播计算完所有参数的梯度后，还需要使用优化方法来更新网络的权重和参数，例如随机梯度下降法(SGD)的更新策略如下：

weight = weight - learning_rate * gradient

在torch.optim中实现大多数的优化方法，例如RMSProp、Adam、SGD等，下面我们使用SGD做个简单的样例

In [22]:
import torch.optim

In [23]:
out = net(input) # 这里调用的时候会打印出我们在forword函数中打印的x的大小
criterion = nn.MSELoss()
loss = criterion(out, y)
#新建一个优化器，SGD只需要要调整的参数和学习率
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)
# 先梯度清零(与net.zero_grad()效果一样)
optimizer.zero_grad() 
loss.backward()

#更新参数
optimizer.step()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


这样，神经网络的数据的一个完整的传播就已经通过PyTorch实现了，下面一章将介绍PyTorch提供的数据加载和处理工具，使用这些工具可以方便的处理所需要的数据。

看完这节，大家可能对神经网络模型里面的一些参数的计算方式还有疑惑，这部分会在第二章 第四节 卷积神经网络有详细介绍，并且在第三章 第二节 MNIST数据集手写数字识别的实践代码中有详细的注释说明。